# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
#HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
#Replace None by your own value and justify
llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0.5)


In [7]:
prompt = ChatPromptTemplate.from_template("Review the following product:{product}")#Write a query that would take a variable to describe any product

In [8]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/d7/f99mzmr12dqbty5ptd50z0dc0000gn/T/ipykernel_1333/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [10]:
product = 'headphones'#Select a product type to be describe
chain.run(product)

'I recently purchased a pair of headphones and I am extremely satisfied with my purchase. The sound quality is excellent, with clear highs and deep bass. The headphones are also very comfortable to wear for extended periods of time, thanks to the cushioned ear cups and adjustable headband.\n\nThe design of the headphones is sleek and modern, and they are lightweight and easy to transport. I also appreciate the built-in microphone and remote control, which allows me to easily take calls and control my music playback without having to take my phone out of my pocket.\n\nOverall, I highly recommend these headphones to anyone in the market for a high-quality, comfortable, and stylish pair of headphones. They have exceeded my expectations and I am very happy with my purchase.'

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
    "Review the following product:{product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    "Expand and improve the following review: {previous_review}. Make it sound more persuasive and engaging."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)
                                            

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
I recently purchased a pair of headphones and I am extremely impressed with the quality and performance. The sound is clear and crisp, with deep bass and excellent treble. The headphones are comfortable to wear for extended periods of time, with soft padding on the ear cups and headband. The noise-cancellation feature works well in blocking out external noise, allowing me to fully immerse myself in my music or audio content. The build quality is also top-notch, with a durable construction that feels like it will last for a long time. Overall, I highly recommend these headphones for anyone looking for a high-quality listening experience.
I recently treated myself to a new pair of headphones and I must say, I am absolutely blown away by the quality and performance of these bad boys. The sound is simply sublime - crystal clear, with a deep, resonating bass that hits just right, and a treble that sparkles like diamonds. 

Not only do these he

"I recently treated myself to a new pair of headphones and I must say, I am absolutely blown away by the quality and performance of these bad boys. The sound is simply sublime - crystal clear, with a deep, resonating bass that hits just right, and a treble that sparkles like diamonds. \n\nNot only do these headphones sound amazing, but they also feel like a dream to wear. The soft padding on the ear cups and headband make them incredibly comfortable, even for extended listening sessions. And let's talk about the noise-cancellation feature - it's like a magic spell that banishes all external noise, allowing you to lose yourself completely in your music or podcasts.\n\nAnd can we talk about the build quality? It's top-notch. These headphones feel like they're built to last - sturdy, durable construction that gives you that confidence that they will be by your side for a long time to come.\n\nIn conclusion, if you're in the market for a pair of headphones that deliver a high-quality liste

**Repeat the above twice for different products**

## SequentialChain

In [21]:
from langchain.chains import SequentialChain

In [16]:
llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  "Translate into {language} this review: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='output_chain_1' #Give a name to your output
                    )


In [17]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    "Summarise the following review:{output_chain_1}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='output_chain_2' #give a name to this output
                    )


In [18]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate into {language_1} this:{output_chain_2}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='output_chain_3'
                      )


In [19]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Follow up the following message:{output_chain_3}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='output_chain_4'
                     )


In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['language','review','language_1'],
    output_variables=['output_chain_1','output_chain_2','output_chain_3','output_chain_4'],
    verbose=True
)

In [24]:
review = df.Review[0]
review

'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.'

In [27]:
inputs = {
    "language": "Spanish",
    "review": "I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases.",
    "language_1": "English"
}

In [28]:
overall_chain(inputs)



> Entering new SequentialChain chain...

> Finished chain.


{'language': 'Spanish',
 'review': 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases.',
 'language_1': 'English',
 'output_chain_1': 'Pedí un juego de cama tamaño king. Mi única crítica sería que me gustaría que el vendedor ofreciera el juego de cama tamaño king con 4 fundas de almohada.',
 'output_chain_2': 'El cliente pidió un juego de cama tamaño king y sugiere que el vendedor ofrezca la opción de incluir 4 fundas de almohada en lugar de solo dos.',
 'output_chain_3': 'The customer requested a king size bedding set and suggests that the seller offer the option of including 4 pillowcases instead of just two.',
 'output_chain_4': 'Thank you for your suggestion! We will definitely consider offering the option of including 4 pillowcases with our king size bedding set in the future. Your feedback is greatly appreciated.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [29]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [30]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [31]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [32]:
llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0)

In [33]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [34]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [35]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [36]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [38]:
print(MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
))


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for answering math questions
History: Go

In [39]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [40]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/d7/f99mzmr12dqbty5ptd50z0dc0000gn/T/ipykernel_1333/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [41]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [42]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [43]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**